In [1]:
import pandas as pd
import numpy as np
import keras
import cv2
import torch
import torchvision.models as models
import os

from os import listdir
from os.path import isfile, join
from tqdm import tqdm
from keras.applications import Xception
from tensorflow.keras.models import Model
from keras.layers import Dense, Activation, Flatten, Dropout

from sklearn.preprocessing import MinMaxScaler

D:\Users\122156\Anaconda3\envs\tensorflow\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\Users\122156\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Users\122156\Anaconda3\envs\tensorflow\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
train_ID_score = pd.read_excel("train_test_split.xlsx", sheet_name = 'train')
test_ID_score = pd.read_excel("train_test_split.xlsx", sheet_name = 'test')

In [3]:
DIR = '.\\influ_xray\\original_data'
training_image = []
training_label = []
testing_image = []
testing_label = []
for img in tqdm(os.listdir(DIR)):
    # train
    if img[:-5] in list(train_ID_score.ID):
        label = float(train_ID_score['score'][train_ID_score['ID'] == img[:-5]])
        image = cv2.imread(os.path.join(DIR,img),cv2.IMREAD_GRAYSCALE)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        image = cv2.resize(image, (299, 299))
        training_image.append(image/255.0)
        training_label.append(label)
        
    # test
    if img[:-5] in list(test_ID_score.ID):
        label = float(test_ID_score['score'][test_ID_score['ID'] == img[:-5]])
        image = cv2.imread(os.path.join(DIR,img),cv2.IMREAD_GRAYSCALE)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        image = cv2.resize(image, (299, 299))
        testing_image.append(image/255.0)
        testing_label.append(label)

100%|██████████████████████████████████████████████████████████████████████████████| 1264/1264 [01:07<00:00, 18.66it/s]


In [4]:
# training_image = torch.from_numpy(np.array(training_image))
# training_label = torch.from_numpy(np.array([[i] for i in training_label]))

In [5]:
base = Xception(
        include_top=True, # 是否包含最後的全連接層 (fully-connected layer)
        weights='imagenet', # None: 權重隨機初始化、'imagenet': 載入預訓練權重
#         input_shape=(256,256,1), # 當 include_top=False 時，可調整輸入圖片的尺寸（長寬需不小於 32）
        )
x = Dense(512, kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4), activation='linear')(base.output)
x = Dropout(0.5)(x)
# x = Dense(128, kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4), activation='linear')(x)
# x = Dropout(0.3)(x)
x = Dense(32, kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4), activation='linear')(x)
x = Dense(1, activation='linear')(x)
model = Model(inputs=base.inputs, outputs=x)

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [7]:
model

In [8]:
# import visualkeras

# visualkeras.layered_view(model).show() # display using your system viewer
# visualkeras.layered_view(model, to_file='output.png') # write to disk
# visualkeras.layered_view(model, to_file='output.png').show() # write and show

# visualkeras.layered_view(model)

In [9]:
model.compile(
#     optimizer=keras.optimizers.Adagrad(learning_rate=0.0005, decay = 0.00001),
#     optimizer=keras.optimizers.SGD(learning_rate=0.0005, decay = 0.00001),
    optimizer=keras.optimizers.Adam(learning_rate=0.0002, decay = 0.00001),
    loss="mean_squared_error",
    metrics=["mse"],
)

In [13]:
history = model.fit(
    np.array(training_image),
    np.array(training_label),
    batch_size=32,
    epochs=1,
    validation_data=((np.array(testing_image), np.array(testing_label))),
    callbacks = [keras.callbacks.EarlyStopping(monitor='mse', patience=5)]
)

23/23 [==============================] - 1067s 46s/step - loss: 1.2003 - mse: 0.9885 - val_loss: 1.6298 - val_mse: 1.4219


In [14]:
model.predict(np.array(testing_image))

array([[2.2949555],
       [3.4782708],
       [1.6996739],
       [1.6443233],
       [4.1444893],
       [1.7334025],
       [4.138855 ],
       [3.9037416],
       [1.6861311],
       [3.9729924],
       [1.6635988],
       [1.8089564],
       [1.6984533],
       [1.656329 ],
       [1.931858 ],
       [1.7191783],
       [1.7881774],
       [4.144295 ],
       [3.3428354],
       [1.6863483],
       [1.6696126],
       [1.7248825],
       [2.8925893],
       [1.6516846],
       [1.717153 ],
       [3.9149873],
       [3.9849775],
       [1.7524743],
       [1.9863223],
       [1.7765553],
       [1.6532377],
       [1.6655862],
       [1.6815242],
       [1.6809248],
       [1.6815709],
       [2.285075 ],
       [2.285075 ],
       [1.6651007],
       [1.6892021],
       [2.4993804],
       [1.6542336],
       [4.147605 ],
       [4.146932 ],
       [1.6666547],
       [1.6817154],
       [1.656486 ],
       [2.7217126],
       [4.1257133],
       [4.0940495],
       [4.04389  ],


In [15]:
#將模型儲存至 HDF5 檔案中
model.save('xception_mse0.9885_val_mse1.4219.hdf5')  # creates a HDF5 file 'my_model.h5'
# 從 HDF5 檔案中載入模型
# model = tf.contrib.keras.models.load_model('my_model.h5')